### <font color=blue>Import Data into MySQL Database</font> 

In [14]:
# Import libraries
import pandas as pd
import pymysql
from pymysql import OperationalError
from sqlalchemy import create_engine
import os

In [15]:
# List of CSV files and their corresponding table names
csv_files = [
    ('hotel_bookings.csv', 'hotel_bookings')
]

In [16]:
# MySQL credentials
user = 'root'
password = 'root'
host = 'localhost'  # or '127.0.0.1'
database = 'hotel_bookings'

# Step 1: Connect to MySQL server (not to a specific database yet)
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    autocommit=True
)

# Step 2: Create a cursor object to interact with the MySQL server
cursor = connection.cursor()

# Step 3: Create the database if it doesn't exist
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
print(f"Connected to {database} database successfully!")

# Step 4: Close the cursor and the connection
cursor.close()
connection.close()

Connected to hotel_bookings database successfully!


In [22]:
# Connect to the MySQL database
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='root',
    database='hotel_bookings'
)
print("==== Connected to MySQL Server ====")
cursor = connection.cursor()


# Get the current working directory
current_dir = os.getcwd()

# Move one level up and then into the 'data' folder
data_folder_path = os.path.join(current_dir, '..', 'data')

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(data_folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f"`{col}` {get_sql_type(df[col].dtype)}" for col in df.columns])
    create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})"
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    connection.commit()

# Close the connection
connection.close()


==== Connected to MySQL Server ====
Processing hotel_bookings.csv
NaN values before replacement:
hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
d